# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [36]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [37]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [38]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [39]:
# To check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Now I am ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [40]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [41]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS 
    sparkifycassandra WITH REPLICATION={'class':'SimpleStrategy','replication_factor':1}""")
    
except Exception as e:
    print(e)
    

#### Set Keyspace

In [42]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifycassandra')
    
except Exception as e:
    print(e)
    

## Create queries to ask the following three questions of the data

   



### Q1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [43]:
# create table artist_song with columns artist, song title, song's length , sessionId and itemInSession with 
# primary key sessionId as a partition key and itemInSession as a clustering key
query = "CREATE TABLE IF NOT EXISTS artist_song"

query = query+"(sessionId INT, itemInSession INT, artist TEXT, song TEXT, length FLOAT, PRIMARY KEY(sessionId,itemInSession) )"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [44]:
# Insert values into artist_song

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song(sessionId , itemInSession , artist , song , length )"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0], line[9],float(line[5])))

In [45]:
# Select data with sessionId = 338, and itemInSession = 4
query = "SELECT artist , song , length FROM artist_song WHERE sessionId = 338 AND itemInSession = 4 "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print([row.artist,row.song,row.length])


['Faithless', 'Music Matters (Mark Knight Dub)', 495.30731201171875]


### Q2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


In [46]:
# create table artist_song_user with columns artist, song , firstName ,lastName, userId , sessionId and itemInSession with 
# primary key userId as a partition key and (sessionId, itemInSession for sorting) as a clustering key
query = "CREATE TABLE IF NOT EXISTS artist_song_user"

query = query+"(userId INT,sessionId INT, itemInSession INT, artist TEXT, song TEXT, firstName TEXT ,\
lastName TEXT,PRIMARY KEY(userId,sessionId,itemInSession) )"

try:
    session.execute(query)
except Exception as e:
    print(e)
                 


                    

In [47]:
# Create table (artist_song_user_for_user_session) for query 2
# The columns are ordered by the parition key, clustering columns then all of the returned values
query = "CREATE TABLE IF NOT EXISTS artist_song_user_for_user_session "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
session.execute(query)

# Insert data into table for query 2
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO artist_song_user_for_user_session (userId, sessionId, artist, song, itemInSession, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))

                    

In [48]:
# Insert values into song_plays_user

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_user(userId, sessionId,artist, song, itemInSession , firstName, lastName )"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))

In [49]:
# Select data with userid = 10, sessionid = 182

query = "SELECT artist , song , firstName , lastName FROM artist_song_user WHERE userId = 10 AND sessionId = 182 "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print([row.artist, row.song, row.firstname, row.lastname])

['Down To The Bone', "Keep On Keepin' On", 'Sylvie', 'Cruz']
['Three Drives', 'Greece 2000', 'Sylvie', 'Cruz']
['Sebastien Tellier', 'Kilometer', 'Sylvie', 'Cruz']
['Lonnie Gordon', 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'Sylvie', 'Cruz']


### Q3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [50]:
# create table song_users with columns  song , firstName ,lastName, userId  with 
# primary key song as a partition key and userId as a clustering key
query = "CREATE TABLE IF NOT EXISTS song_users"

query = query+"(song TEXT, userId INT, firstName TEXT , lastName TEXT , PRIMARY KEY(song , userId) )"

try:
    session.execute(query)
except Exception as e:
    print(e)
                

                    

In [51]:
# Insert values into songs_of_user

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_users(song , userId , firstName  , lastName )"
        query = query + "VALUES(%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]),line[1], line[4]))

In [52]:
# Select individuals who listened to the song 'All Hands Against His Own'

query = "SELECT firstName , lastName FROM song_users WHERE song = 'All Hands Against His Own' "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print([row.firstname, row.lastname])

['Jacqueline', 'Lynch']
['Tegan', 'Levine']
['Sara', 'Johnson']


### Drop the tables before closing out the sessions

In [53]:
# Drop artist_song, artist_song_user and song_users tables
session.execute("DROP TABLE artist_song")
session.execute("DROP TABLE artist_song_user")
session.execute("DROP TABLE song_users")

### Close the session and cluster connection¶

In [54]:
session.shutdown()
cluster.shutdown()